In [5]:
# by Joh Schöneberg 2018

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from numpy import genfromtxt


import pandas as pd

from mpl_toolkits import mplot3d
#%matplotlib notebook

import pylattice

import pylattice.functions.InputProcessing as inproc
import pylattice.functions.TiffProcessing as tiffproc
import pylattice.classes.

# input data

In [7]:
def getInputParameter(inputParametersPandas,key):
    #this locates the row, gets the result out of its array form and strips whitespaces away
    return (((inputParametersPandas.loc[inputParametersPandas['key'] == key]).values)[0,1]).strip()

In [6]:
inputParameters = pd.read_csv('../_inputParameters.csv',names=['key','value'])
inputParameters


,key,value
0,inputDataFolder,/Users/johannesschoeneberg/git/LatticeTrack/d...
1,outputDataFolder,/Users/johannesschoeneberg/git/LatticeTrack/d...
2,movieLength,20
3,allowedMaxNumDetectionsPerFrame,3000
4,framerate_msec,5689
5,sigma_detectionLoG,2
6,ch0_outputDataFolder,output_488
7,ch0_uniqueFilenameString,_488_
8,ch0_detectionFilename,Detection3D.mat
9,ch0_trackingFilename,trackedFeatures.mat


In [17]:
inputDataFolder = getInputParameter(inputParameters,"inputDataFolder")
outputDataFolder = getInputParameter(inputParameters,"outputDataFolder")
ch0_outputDataFolder  = getInputParameter(inputParameters,"ch0_outputDataFolder")
ch0_trackingCsvFilename = getInputParameter(inputParameters,"ch0_trackingCsvFilename")
ch1_outputDataFolder  = getInputParameter(inputParameters,"ch1_outputDataFolder")
ch1_trackingCsvFilename = getInputParameter(inputParameters,"ch1_trackingCsvFilename")

ch0_uniqueFilenameString = getInputParameter(inputParameters,"ch0_uniqueFilenameString")
ch1_uniqueFilenameString = getInputParameter(inputParameters,"ch1_uniqueFilenameString")
ch1_trackColor = getInputParameter(inputParameters,"ch1_trackColor")
ch1_trackingBildFilename = getInputParameter(inputParameters,"ch1_trackingBildFilename")
#framerate_msec = float(getInputParameter(inputParameters,"framerate_msec"))
movieLength = int(getInputParameter(inputParameters,"movieLength"))
print(ch1_trackColor)

purple


In [18]:
data = pd.read_csv(outputDataFolder+'/'+ch1_trackingCsvFilename,header=0)
data.columns = ["trackId", "tracklength", "frameId", "particleId", "x", "y", "z", "A", "noIdea1", "noIdea2", "noIdea3", "noIdea4"]
data[0:5]

,trackId,tracklength,frameId,particleId,x,y,z,A,noIdea1,noIdea2,noIdea3,noIdea4
0,1,20,1,11,6.102233e+01,2.826133e+01,1.882295e+01,2.133890e+04,6.346962e-02,6.354327e-02,6.350296e-02,6.273288e+02
1,1,20,2,12,6.000385e+01,2.809711e+01,1.717730e+01,2.579957e+04,6.135022e-02,6.136002e-02,6.138305e-02,7.336236e+02
2,1,20,3,0,5.769674e+01,2.830042e+01,1.725752e+01,2.482572e+04,5.158390e-02,5.121466e-02,5.119355e-02,5.845128e+02
3,1,20,4,0,5.755685e+01,2.847145e+01,1.759348e+01,2.629457e+04,5.153810e-02,5.150935e-02,5.157791e-02,6.185938e+02
4,1,20,5,0,5.817331e+01,2.840556e+01,1.755592e+01,2.306101e+04,5.080201e-02,5.109420e-02,5.105263e-02,5.388592e+02


# select subset

In [32]:
trackIdsLong = (data[data['tracklength'] > 10]).drop_duplicates(subset='trackId')['trackId'].values
allTrackIds = data['trackId'].values


# parse through the track class first

In [25]:
class TrackRaw:
    def __init__(self,pandasTrackData):
        tracklength = int((pandasTrackData['tracklength'].values)[0])
        
        #trim the end of the track until you got rid of all the nans
        xCoordLastEntry = pandasTrackData[tracklength-1:tracklength]['x'].astype(float).values
        #print(xCoordLastEntry,tracklength)
        while np.isnan(xCoordLastEntry):
            tracklength = tracklength-1
            xCoordLastEntry = pandasTrackData[tracklength-1:tracklength]['x'].astype(float).values
            #print(xCoordLastEntry,tracklength)
        
        track = pandasTrackData[0:tracklength] # this function kills all the NaNs that come from matlab
        
        self.id = track['trackId'].astype(int).values[0]
        self.len = tracklength

        self.coords = track[['x','y','z']].astype(float).values
        
            
        self.cm = np.nanmean(self.coords,axis=0)
        self.maxDist = np.linalg.norm(self.coords[0]-self.coords[-1])
        
        self.particleIDs = track['particleId'].astype(int).values
        self.A = track['A'].astype(float).values
        self.Amean = np.nanmean(self.A)

        self.frameIDs = track['frameId'].astype(int).values
        
    def reveal(self):
        print('id',self.id)
        print('tracklength',self.len)
        print('center of mass',self.cm)
        print('coords',self.coords)
        print('particleIDs',self.particleIDs)
        print('A',self.A)
        print('frameIDs',self.frameIDs)

           
    def plot(self):
        plt.figure(dpi=300)
        ax = plt.axes(projection='3d')
        ax.plot3D(self.coords[:,0], self.coords[:,1], self.coords[:,2], 'grey')
        ax.scatter3D(self.coords[:,0], self.coords[:,1], self.coords[:,2],c=self.A, cmap='plasma',s=100);
        plt.xlabel('x [px]')
        plt.ylabel('y [px]')
        
    def writeBILD(self,BILDfilename,color='black',center=[]):
        filename=BILDfilename
        file = open(BILDfilename,'w') 

        file.write(".transparency 0.5\n")
        file.write(".color "+color+"\n")
    
        line = ".comment trackID"+str(self.id)+"\n"
        file.write(line)
        

        
        for i in range(1,self.len):
            tzero = self.coords[i-1]
            tone = self.coords[i]
            if len(center) != 0:
                tzero = tzero-center
                tone = tone-center

            
    
            # Data for a three-dimensional line
            x0 = float(tzero[0])
            y0 = float(tzero[1])
            z0 = float(tzero[2])
            A0 = float(self.A[i-1])
            
            x1 = float(tone[0])
            y1 = float(tone[1])
            z1 = float(tone[2])
            A1 = float(self.A[i])
            
            if(math.isnan(x0) or math.isnan(y0) or math.isnan(z0) or math.isnan(x1) or math.isnan(y1) or math.isnan(z1)):
                line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
                print(line)
                file.write(".comment "+line)
                continue
                    
            line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
            file.write(line)
        
        file.close()

In [33]:
import timeit


trackIdListInvestigated = trackIdsLong

start_time = timeit.default_timer()

tracks = []
cm0 = []
len0 = []
Amean0 = []
maxDist0 = []



for i in range(0,len(trackIdListInvestigated)):
    if(i %1000 == 0):
        print(str(i)+"/"+str(len(trackIdListInvestigated)))
    a = TrackRaw(data[data['trackId'] == trackIdListInvestigated[i]])
    tracks.append(a)
    cm0.append(a.cm)
    len0.append(a.len)
    Amean0.append(a.Amean)
    maxDist0.append(a.maxDist)
    
elapsed = timeit.default_timer() - start_time
print('time elapse: '+str(np.round(elapsed,decimals=2))+'s')

0/70
time elapse: 0.17s


In [35]:
filename=ch1_trackingBildFilename
path = outputDataFolder+'/'+filename
print(path)

/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/tracks.bild


In [37]:
for track in tracks:
    numberOfZeros = int(np.ceil(np.log10(movieLength+1)))
    output_filename = 'track_'+str(track.id).zfill(numberOfZeros)+'.bild'
    path = outputDataFolder+'/'+output_filename
    print(path)
    track.writeBILD(path,color='white')

/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_01.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_02.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_03.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_04.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_05.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_06.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_07.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_08.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_09.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_10.bild
/Users/johannesschoe

# directly from the csv file

In [19]:
len(data)

2573

In [20]:
longtracks = data[data['tracklength'] >= 20]
print(len(longtracks))
print(len(np.unique(data['trackId'].values)))
print(np.unique(data['trackId'].values))

1140
324
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 2

In [39]:
#plot them all
import math

data = data

drawColor = ch1_trackColor
drawColor = 'white'

filename=ch1_trackingBildFilename
file = open(outputDataFolder+'/'+filename,'w') 

file.write(".transparency 0.5\n")
file.write(".color "+drawColor+"\n")

for trackID in np.unique(data['trackId'].values):
    
    line = ".comment trackID"+str(trackID)+"\n"
    file.write(line)
    
    
    track = data[data['trackId'] == trackID]
    tracklength = track['tracklength'].iloc[0]
    
    for i in range(1,tracklength):
        tzero = track.iloc[i-1]
        tone = track.iloc[i]
        

        # Data for a three-dimensional line
        x0 = float(tzero['x'])
        y0 = float(tzero['y'])
        z0 = float(tzero['z'])
        A0 = float(tzero['A'])
        
        x1 = float(tone['x'])
        y1 = float(tone['y'])
        z1 = float(tone['z'])
        A1 = float(tone['A'])
        
        if(math.isnan(x0) or math.isnan(y0) or math.isnan(z0) or math.isnan(x1) or math.isnan(y1) or math.isnan(z1)):
            line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
            print(line)
            continue
        
        
        #.arrow x1 y1 z1 x2 y2 z2 [r1 [r2 [rho]]] 
        
        line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
        file.write(line)
        
file.close()

.arrow 79.10506 54.10194 29.24221 nan nan nan

.arrow nan nan nan 79.31233 54.34717 28.74783

.arrow 65.12139 75.89871 55.29878 nan nan nan

.arrow nan nan nan 64.72372 76.97292 54.76919

.arrow 5.569371 25.25257 56.60967 nan nan nan

.arrow nan nan nan 5.526814 25.06624 57.21887

.arrow 4.632 24.80286 57.13399 nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan 5.651869 26.627 57.08865

.arrow 21.73191 41.73386 57.18824 nan nan nan

.arrow 57.54178 17.43141 63.94101 nan nan nan

.arrow nan nan nan 55.60775 18.50711 64.72944

.arrow 55.77796 17.99546 64.57734 nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan 54.54417 17.7571 64.55712

.arrow 10.34043 48.31249 63.70354 nan nan nan

.arrow nan nan nan 7.263132 47.64709 63.34926

.arrow 7.263132 47.64709 63.34926 nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan 8.972535 47.20685 63.82314

.arrow 75.84109 77.38692 16.96338 nan nan nan

.arrow nan nan nan nan nan nan

.arrow nan nan nan nan nan nan



## Do regular plotting